<a href="https://colab.research.google.com/github/somarbilal/Arabic-Sentiment-Analysis/blob/main/Text_Classification_Arabic_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import csv
import re
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
import itertools
from pandas import DataFrame
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [ ]:
files = pd.read_csv("https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/a59d0a2ea9061483935309799904e8bfec0e8c57/datasets/RES1.csv")

In [ ]:
print(files)

      polarity                                               text  \
0           -1  اولا: المنيو تغير الشورما اصبحت اعتياديه بأختف...   
1           -1  من محلات الشاورما ذات الشعبيه لتميز الصلصات ال...   
2            1  دجاج طازج يحضر امامك على الطلب لا يقوم باعدة ا...   
3            1  فكما تعرف أستراليا بالكنغر والكوالا. فإنها تعر...   
4           -1  إسمحو لي أن أقيم مطعم هاشم بصفتي فلسطيني عشت ف...   
...        ...                                                ...   
8359         1  أتمنى تعملو بطاقات خاصة أو ذهبية للعملاء الدائ...   
8360         1  المشويات عندهم اطيب مشويات اكلتها بحياتي والاس...   
8361         1  المطعم ممتاز للغايه ، من ناحيه تجهيزات المطعم ...   
8362        -1  جينا عندهم الساعة 10 بالليل مافى احد اتبرع يشو...   
8363         1  اذا كان اللي بالشارع التجاري اوووه ماشاء الله ...   

      restaurant_id  user_id  
0               296      423  
1               296      423  
2              5027    39580  
3               642      444  
4               

In [ ]:
print(files.head())

   polarity                                               text  restaurant_id  \
0        -1  اولا: المنيو تغير الشورما اصبحت اعتياديه بأختف...            296   
1        -1  من محلات الشاورما ذات الشعبيه لتميز الصلصات ال...            296   
2         1  دجاج طازج يحضر امامك على الطلب لا يقوم باعدة ا...           5027   
3         1  فكما تعرف أستراليا بالكنغر والكوالا. فإنها تعر...            642   
4        -1  إسمحو لي أن أقيم مطعم هاشم بصفتي فلسطيني عشت ف...            434   

   user_id  
0      423  
1      423  
2    39580  
3      444  
4     2191  


In [ ]:
df=pd.read_csv("https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/a59d0a2ea9061483935309799904e8bfec0e8c57/datasets/RES1.csv")
del df['user_id']
del df['restaurant_id']
        
df.to_csv('newwwwwwww.csv')


In [ ]:
df.keys()

Index(['polarity', 'text'], dtype='object')

In [ ]:
print(df.head())

   polarity                                               text
0        -1  اولا: المنيو تغير الشورما اصبحت اعتياديه بأختف...
1        -1  من محلات الشاورما ذات الشعبيه لتميز الصلصات ال...
2         1  دجاج طازج يحضر امامك على الطلب لا يقوم باعدة ا...
3         1  فكما تعرف أستراليا بالكنغر والكوالا. فإنها تعر...
4        -1  إسمحو لي أن أقيم مطعم هاشم بصفتي فلسطيني عشت ف...


In [ ]:
print(df)

      polarity                                               text
0           -1  اولا: المنيو تغير الشورما اصبحت اعتياديه بأختف...
1           -1  من محلات الشاورما ذات الشعبيه لتميز الصلصات ال...
2            1  دجاج طازج يحضر امامك على الطلب لا يقوم باعدة ا...
3            1  فكما تعرف أستراليا بالكنغر والكوالا. فإنها تعر...
4           -1  إسمحو لي أن أقيم مطعم هاشم بصفتي فلسطيني عشت ف...
...        ...                                                ...
8359         1  أتمنى تعملو بطاقات خاصة أو ذهبية للعملاء الدائ...
8360         1  المشويات عندهم اطيب مشويات اكلتها بحياتي والاس...
8361         1  المطعم ممتاز للغايه ، من ناحيه تجهيزات المطعم ...
8362        -1  جينا عندهم الساعة 10 بالليل مافى احد اتبرع يشو...
8363         1  اذا كان اللي بالشارع التجاري اوووه ماشاء الله ...

[8364 rows x 2 columns]


#preprocessing

In [ ]:
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
 
    text = re.sub(noise, '', text)
    text = ''.join(char for char, _ in itertools.groupby(text))#منع تكرار الأحرف
    return text

In [ ]:
print(normalize("خيبت ظني ّ💔 🔕"))

خيبت ظني 💔 🔕


In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    #print (text)
    text= emoji_pattern.sub(r'', text)
    text = ''.join(char for char, _ in itertools.groupby(text))#remove white space
    return text

In [ ]:
print(remove_emoji("كم ﻋرفنآ ﻧآس ﻣﺂفيھآ ۆفآء 💔 🔕💔 🔕💔 🔕.."))

كم رفنآ آس فيھآ ۆفآء .


**STOP WORD REMOVE**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ar_stop_word_list.txt to ar_stop_word_list (1).txt


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def StopWordRemove(text):
    ar_stop_list=open("ar_stop_word_list.txt","r",encoding='utf-8')
    stop_words=ar_stop_list.read().split('\n')
    needed_words=[]
    words=word_tokenize(text)
    for w in words:
        if w not in (stop_words):
            needed_words.append(w)
    filtered_sentence=" ".join(needed_words)
    return filtered_sentence

In [ ]:
print(StopWordRemove(" هذا الرجل ذو شأن عظيم"))

الرجل شأن عظيم


**stemming**

In [ ]:
def stemming(text):
    st= ISRIStemmer()
    stemmed_words=[]
    words=word_tokenize(text)
    for w in words:
        stemmed_words.append(st.stem(w))
    stemmed_sentence=" ".join(stemmed_words)
    return stemmed_sentence
        

In [ ]:
print(stemming("بسم الله الرحمن الرحيم"))

بسم الل رحم رحم


In [ ]:
def prepareDatasets(df):
    sentences=[]
    for index, r in df.iterrows():
        text=StopWordRemove(r['text'])
        text=normalize(r['text'])
        text=stemming(r['text'])
        
        if r['polarity']==-1:
            sentences.append([text,'neg'])
        else:
            sentences.append([text,'pos'])
            
    df_sentences=DataFrame(sentences, columns=['text','polarity'])
    return df_sentences

In [ ]:
preprocessed_reviews=prepareDatasets(df)

In [ ]:
preprocessed_reviews

,text,polarity
0,اول : نيو تغر شور صبح اعتياديه أختفاء صلص خير ...,neg
1,من محل شور ذات شعب تمز صلص قدم عها يمكن خير شو...,neg
2,دجج طزج حضر ممك على طلب لا يقم بعد تسخ تخذ بشر...,pos
3,فكم عرف رال كنغر كلا . فإن عرف ايض أكل مثل فيج...,pos
4,محو لي ان اقم طعم هشم بصف لسط عشت في ارد 24 عا...,neg
...,...,...
8359,منى عملو بطق خصة او ذهب عملاء دئم يكون لهم وضع...,pos
8360,شوي عند اطب شوي اكل بحا قبل مرة حلو نظافةةةةةة...,pos
8361,طعم متز غيه ، من نحي جهز طعم ، ويض من نحي أكل ...,pos
8362,جين عند سعة 10 ليل افى احد برع شوفل طال فضي جل...,neg


**feature Extraction**

In [ ]:
def featureExtraction(data):
    vectorizer= TfidfVectorizer(min_df=10, max_df=0.75, ngram_range=(1,3))
    tfidf_data=vectorizer.fit_transform(data)
    return tfidf_data

In [ ]:
def learning(clf, x, y):
    x_train, x_test, y_train, y_test=train_test_split(x,y, test_size=.4, random_state=43)
    
    classifier=clf()
    classifier.fit(x_train, y_train)
    predict= cross_val_predict(classifier, x_test, y_test, cv=10)
    
    scores= cross_val_score(classifier, x_test, y_test, cv=10)
    
    print(scores)
    print("Accuracy of %s: %0.2f (+/- %0.2f)"%(classifier,scores.mean(), scores.std()*2))
    print(classification_report(y_test,predict))
    

In [ ]:
def main(clf):
    df=pd.read_csv("https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/a59d0a2ea9061483935309799904e8bfec0e8c57/datasets/RES1.csv")
    del df['user_id']
    del df['restaurant_id']
    preprocessed_reviews=prepareDatasets(df)
    
    data, target=preprocessed_reviews['text'],preprocessed_reviews['polarity']
    
    tfidf_data=featureExtraction(data)
    
    learning(clf, tfidf_data, target)

In [ ]:
main(MultinomialNB)

[0.74925373 0.77910448 0.8        0.82985075 0.79402985 0.76716418
 0.78742515 0.7994012  0.77245509 0.79341317]
Accuracy of MultinomialNB(): 0.79 (+/- 0.04)
              precision    recall  f1-score   support

         neg       0.94      0.29      0.44       976
         pos       0.77      0.99      0.87      2370

    accuracy                           0.79      3346
   macro avg       0.86      0.64      0.65      3346
weighted avg       0.82      0.79      0.74      3346



In [ ]:
clfs=[MultinomialNB, BernoulliNB,LogisticRegression, SGDClassifier,SVC, LinearSVC, NuSVC]

In [ ]:
for clf in clfs:
    main(clf)

[0.74925373 0.77910448 0.8        0.82985075 0.79402985 0.76716418
 0.78742515 0.7994012  0.77245509 0.79341317]
Accuracy of MultinomialNB(): 0.79 (+/- 0.04)
              precision    recall  f1-score   support

         neg       0.94      0.29      0.44       976
         pos       0.77      0.99      0.87      2370

    accuracy                           0.79      3346
   macro avg       0.86      0.64      0.65      3346
weighted avg       0.82      0.79      0.74      3346

[0.75820896 0.75223881 0.76119403 0.7641791  0.73731343 0.78208955
 0.80538922 0.77844311 0.79041916 0.75748503]
Accuracy of BernoulliNB(): 0.77 (+/- 0.04)
              precision    recall  f1-score   support

         neg       0.63      0.50      0.56       976
         pos       0.81      0.88      0.84      2370

    accuracy                           0.77      3346
   macro avg       0.72      0.69      0.70      3346
weighted avg       0.76      0.77      0.76      3346

[0.78507463 0.8        0.8268656